# **Explanatory Data Analysis: Helping Erin Robinson Achieve Her Objective**

## **Introduction**
This project aims to assist Erin Robinson, a socially responsible investor, in identifying the most profitable real estate opportunities in Socioeconomically Challenged Districts (SDCs). Erin's objective is to invest in properties within underprivileged neighborhoods, ensuring her investments not only yield a modest profit but also contribute positively to the community. 

By analyzing sales and property data, I explored key trends and identified the top 10 properties Erin should consider purchasing. These recommendations are supported by detailed insights, visualizations, and a structured approach to data preparation and analysis.

---

## **Table of Contents**
1. [Data Exploration & Cleaning](#data-exploration--cleaning)
2. [General Data: Insights & Graphs](#general-data-insights--graphs)
3. [SDC Data: Insights & Graphs](#sdc-data-insights--graphs)
4. [Top 10 Suggestions: Best Houses for Erin](#top-10-suggestions-best-houses-for-erin)

# **Data Exploration and Cleaning**

## **Overview**
This section focuses on preparing and cleaning the data to ensure accurate and insightful analysis. The steps include importing necessary libraries, exploring the dataset for inconsistencies, cleaning and transforming the data, and merging relevant tables for further analysis.

---

## **Steps**
1. **Importing Libraries**:
    - Utilized essential Python libraries such as:
      - **Pandas**: For data manipulation and cleaning.
      - **Seaborn**: For creating visualizations.
      - **Plotly**: For interactive and dynamic graphs.

2. **Data Cleaning and Exploration**:
    - Checked for null values, duplicates, and inconsistencies.
    - Handled missing or invalid data to ensure the dataset is clean and ready for analysis.

3. **Merging Tables**:
    - Combined multiple datasets to create a unified table for comprehensive analysis.
    - Ensured proper alignment and removal of irrelevant columns during the merge process.


In [ ]:
# Setting the scene: First we make sure we have all libraries we need

import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format # to set the format clean with only 2 decimals in float64

# Making the CSV data available for analysis

df_det_csv = pd.read_csv("data/details.csv")
df_sal_csv = pd.read_csv("data/sales.csv")

In [ ]:
# DataFrame & Getting the Sense of Data

# Details Table 

df_det = pd.DataFrame(df_det_csv)

# null / NaN values
# outliers
    

In [ ]:
# Details Table

# convert id and zipcode into string

df_det = df_det.astype({"id": str})
df_det = df_det.astype({"zipcode": str})

# drop latitude and longtitude as they are not necessary for my analysis

df_det_1 = df_det.copy()
df_det_1 = df_det_1.drop(columns =["lat", "long"])

# convert the year content into date and time

df_det_1["yr_built"] = pd.to_datetime(df_det_1["yr_built"], format ='ISO8601').dt.year
df_det_1["yr_renovated"] = pd.to_datetime(df_det_1["yr_built"], format ='ISO8601').dt.year

# null value analysis

df_det_1.isna().sum() # waterfront, view and sqft_basement values are to be watched out. 

# new columns:

# loft - mezzanine column

def half_units(x):
    if x % 1 == 0.5:
        return 'Split-Level'
    else:
        return 'Regular'
        

df_det_1["Floor_Design"] = df_det_1["floors"].apply(half_units)

df_det_1.head()

# bathroom capacity column 

def bath_type(x):
    if x % 1 == 0.25:
        return 'has Mini Toilet'
    elif x % 1 == 0.5:
        return 'has Toilet'
    elif x % 1 == 0.75:
        return 'has Bathroom'
    else:
        return 'Full Bathroom'

df_det_1["Bathroom_Type"] = df_det_1["bathrooms"].apply(bath_type)

# the last version

details = df_det_1
details.head()

In [ ]:
# Sales Table

df_sal = pd.DataFrame(df_sal_csv)

# Date & String Formatting

df_sal_1 = df_sal.copy()
df_sal_1["date"] = pd.to_datetime(df_sal["date"], format ='ISO8601')
df_sal_1["year"] = df_sal_1["date"].dt.year
df_sal_1["month"] = df_sal_1["date"].dt.month
df_sal_1 = df_sal_1.astype({"house_id": str})
df_sal_1 = df_sal_1.astype({"id": str})

# Changing the Name

sales = df_sal_1
sales


# **General Data: Insights & Graphs**

## **Overview**
This section explores overall data trends and visualizations to gain a deeper understanding of the dataset. Key analyses include:

---

## **Steps**
1. **Basic Correlation Analysis**:
    - Examined the relationship between price and attributes such as grade, condition, and square footage.

2. **Identifying Socioeconomically Challenged Districts (SDCs)**:
    - Defined the 15 SDCs based on key indicators.
    - Analyzed their sales levels to focus on opportunities in underprivileged areas.

3. **Visualizations**:
    - Created correlation heatmaps and scatter plots to support findings.


In [ ]:
# A single table concatenation 

sales_details = details.merge(sales, how="left", left_on="id", right_on="house_id").reset_index()

# Data Cleaning: get rid of null data 

sales_details_v1 = sales_details[sales_details["view"].isna()==False]
sales_details_v2 = sales_details_v1[sales_details_v1["waterfront"].isna()==False]
sales_details_v3 = sales_details_v2[sales_details_v2["sqft_basement"].isna()==False].reset_index()

# get rid of high decimals by turning them into the integer

sdv3 = sales_details_v3 
sdv3 = sdv3.astype(
    {'bedrooms': int, 
     'sqft_living': int,
     'sqft_living': int,
     'sqft_lot': int,
     'waterfront': int,
     'view': int,
     'sqft_living15': int,
     'sqft_lot15': int,
     'price': int
    })

#cleaned table

sdf = sdv3.iloc[:, 2:].rename({"id_x": "opportunity_id", "id_y": "sales_id"}, axis=1)


In [ ]:
# What values are correlated to the price overall?

plt.rcParams.update({'font.size': 8}) # setting the font size 8 by default.

fig, ax = plt.subplots(figsize=(10,6)) 
sns.boxplot(sdf, x="grade", y="price", hue="condition", palette="Set2", ax=ax, showmeans=True, showfliers=False) # grade and condition is correlated to the price.
plt.show() 

fig, ax = plt.subplots(figsize=(6,6))
sns.boxplot(sdf, x="view", y="price", ax=ax, showmeans=True, showfliers=False) # view count is correlated to the price.
plt.show() 

fig, ax = plt.subplots(figsize=(6,6))
sns.boxplot(sdf, x="waterfront", y="price", ax=ax, showmeans=True, showfliers=False) #waterfront location is correlated to the price.
plt.show() 

# Hypothesis 1: Grade, condition, view and waterfront are correlated to the price







In [ ]:
# group it by the zipcode and bring the most correlated values 

sdf_zip1 = sdf.groupby("zipcode")
sdf_zip2 = sdf_zip1[["zipcode", "price", "condition", "grade"]].mean(numeric_only=True) #numeric_only = True

# create small dataframes by each attribute to see the correlation

sdf_zip_price = sdf_zip2.sort_values("price").head(15)

# bar plots to demonstrate the relationship between the metrics & top poor neighborhoods

fig, ax1 = plt.subplots(figsize=(10,6))

# main graph with the price

sns.barplot(sdf_zip_price, x="zipcode", y="price", ax=ax1, label="Prices") #correlated
ax1.set_ylabel('House Prices', color="black")
ax1.tick_params(axis='y', labelcolor="black")

# additional graph with the grade
ax2 = ax1.twinx() # add a new line, the 2nd new line y axis
sns.lineplot(sdf_zip_price, x="zipcode", y="grade", marker='o', color='red', ax=ax2, label="Grade")
ax2.set_ylabel('Grading System', color="black")
ax2.tick_params(axis='y', labelcolor='black')

plt.title("Bar Chart with Prices & Grades by the Zipcodes")

ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
ax1.set_ylim(200000,400000)
ax2.set_ylim(6,8)
plt.xticks(rotation=45)
plt.style.use('bmh')
plt.show() 

# Assumption 1: The bottom 15 neighborhoods with the lowest average sales prices would lead to a socially responsible buy. 


# **SDCs: Insights & Graphs**

## **Overview**
This section examines housing trends in Socioeconomically Challenged Districts (SDCs), focusing on:

1. **Price Development**:
   - Analyzed overall house price trends in the last 12 months.

2. **Twice-Sold Houses**:
   - Investigated their impact on the price surge.

3. **Profit Margins**:
   - Explored the characteristics of profit margins unique to SDCs.


In [ ]:
poverty = sdf_zip_price.reset_index()["zipcode"]
hp = sdf[sdf["zipcode"].isin(poverty)] #neighborhoods in poverty that we will focus on

# sales trends in the poor neighborhoods - Plotly

hp_monthly_sales = hp.groupby(["year", "month"])["price"].mean().reset_index()
hp_monthly_sales["Year-Month"] = hp_monthly_sales["year"].astype(str) + '-' + hp_monthly_sales["month"].astype(str)
hp_monthly_sales

fig = px.line(hp_monthly_sales, x="Year-Month", y="price", markers=True)

fig.update_layout(
    title = {"text": "Sales Trends in the Poor N.Hoods in the Past 12 Months",
             "x":0.5,
             "xanchor": "center",
             "yanchor": "top"
            },
    xaxis_title = "Months",
    yaxis_title = "House Prices"
)

fig.update_layout(
    title={
        'text': "Monthly Average House Prices",
        'x': 0.5,  # Center-align the title
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20}
    }
)


fig.show()

# Hypothesis 2: The House Prices are Increasing in the Socioeconomically Challenged Districts (SCD).


In [ ]:
# number of opportunities sold twice or more
hpp = hp.groupby('opportunity_id').filter(lambda x: x['sales_id'].nunique() > 1) 

hpd = hp[hp["opportunity_id"].isin(hpp["opportunity_id"])] #2+ sold opps list
hpd = hpd.sort_values(["opportunity_id", "date"]) # sorting them by opp id and the date
hpd["sales_time"] = hpd.groupby("opportunity_id").cumcount() + 1 # sales time function to pivot the table
hpdp = hpd.pivot(index="opportunity_id", columns="sales_time", values="price") #pivot table to observe the difference
hpdp.columns= ["1st Sales", "2nd Sales"] # new column names 
hpdp["profit_%"] = round((hpdp["2nd Sales"] / hpdp["1st Sales"] - 1) * 100, ndigits=None) # building a new column for the profit margin
hpdpp = hpdp.sort_values("profit_%", ascending=False).head(20) #sorting the table based on the profit margin

salesprofit = pd.DataFrame(hpdpp).reset_index() #adding it into the dataframe
salesprofit

#Graph of the Top 20 profitable houses driving strong profit

fig1 = px.bar(salesprofit,
                 x = "opportunity_id",
                 y = "profit_%",
                 color_continuous_scale = "Plasma"
                )

fig1.update_layout(
                    title={
                            'text': "Profit Margin of the Top 20 Houses",
                            'x': 0.5,  # Center-align the title
                            'xanchor': 'center',
                            'yanchor': 'top',
                            'font': {'size': 20}},
                    xaxis_title = "Houses",
                    yaxis_title = "Profit Margin"
                                                    )
fig1.show()

# Hypothesis 3: Twice sold houses have significant contribution in the house price bump in the last 12 months.



In [ ]:
# grade, condition, view, waterfront: are they also the drivers for profit in Socioeconomically Challenged Districts?

# create a subtable to join with the salesprofit table

hpsub = hp[["opportunity_id",
           "condition",
           "grade",
           "view",
           "bathrooms",
           "sqft_living",
           "waterfront"]].groupby("opportunity_id").mean()

# join tables - to analyze the correlation

hpc = salesprofit.merge(hpsub, how="left", left_on="opportunity_id", right_on="opportunity_id").reset_index()
hpc

# initial comments: no waterfront, no views, reverse correlated with "condition, grade"

fig10 = px.density_heatmap(hpc,
                           x = "grade",
                           y = "condition",
                           z = "profit_%",
                           color_continuous_scale = 'RdBu',
                  )

fig10.update_layout(
                        title={
                            'text': "Profit vs. Quality",
                            'x': 0.5,  # Center-align the title
                            'xanchor': 'center',
                            'yanchor': 'top',
                            'font': {'size': 20}},
    xaxis_title="Grade",
    yaxis_title="Condition"
)

fig10.show()

# Hypothesis 4: Profit and overall Quality (grade & condition) are reversed correlated to each other.

In [ ]:
# Hypothesis 5: Top 20 houses are not viewed at all & by the waterfront. 

# pie chart for the Top 20 twice-sold houses and their waterfront attribute.

fig11 = px.pie(hpc, "waterfront")

fig11.update_layout(showlegend = False,
                        title={
                            'text': "Share of Houses not by the Waterfront",
                            'x': 0.5,  # Center-align the title
                            'xanchor': 'center',
                            'yanchor': 'top',
                            'font': {'size': 20}}
)
fig11.show()

# pie chart for the Top 20 twice-sold houses and their viewcount attribute.

fig12 = px.pie(hpc, "view")

fig12.update_layout(showlegend = False,
                        title={
                            'text': "Share of Houses not Viewed",
                            'x': 0.5,  # Center-align the title
                            'xanchor': 'center',
                            'yanchor': 'top',
                            'font': {'size': 20}}
)
fig12.show()

# **Top 10 Suggestions: Best Buys for Erin**

Assuming the once-sold, low grade and condition houses with no active views and not by the waterfront, here are the top 10 buys for Erin.


In [ ]:
#Assumption: #2 - Sold Once, no Waterfront, not Viewed yet, (avg(Grade + Condition))

# create a table with the houses only sold once in the SCDs

hponce = hp.groupby('opportunity_id').filter(lambda x: x['sales_id'].nunique() == 1) #table filter
hponce["grade_cond"] = hponce["grade"] * 0.5 + hponce["condition"] * 0.5  #overall metric combining grade and condition to sort the top choices alongside the price
hp_suggest = hponce.query("view == 0 and waterfront == 0").sort_values(["price", "grade_cond"]) #waterfront and view filters

#top 10: no waterfront, no view, lowest condition & grade in the cohort, cheapest price
top10buy = hp_suggest[["opportunity_id", "price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot", "condition"]].rename({'opportunity_id':'recommended_house'}).head(10) 

top10buy["bathrooms"] = top20buy["bathrooms"].round(2)  # Round bathrooms to 2 decimal places
t10b = top10buy.reset_index() # resetting index
t10b.index+=1 #starting from 1
t10b.iloc[:,1:] #final shape of top 10





